# Python Class Hierachy Practice

The goal of the excercise is to write a class for logical circuits. The visual map of the hierachy is as follows:

- LogicGate
    - BinaryGate, a gate with 2 inputs/pins (is-a LogicGate)
        - AndGate (is-a UnaryGate)
        - XorGate (is-a UnaryGate)
        - NandGate (is-a UnaryGate)
        - NorGate (is-a UnaryGate)
    - UnaryGate (is-a LogicGate)
        - NotGate (is-a UnaryGate)
        - TermGate, terminal gate for input (is-a UnaryGate)

- Connector (has 2 LogicGates): this connects two LogicGates and faciliates signal propagation
    

## Parent LogicGate

In [1]:
class LogicGate:
    
    def __init__(self, n):
        self.label = n
        self.output = None
        
    def getName(self):
        return self.label
    
    def getOutput(self):
        self.output = self.performGateLogic()
        return self.output

## Child Binary/UnaryGate

In [2]:
class BinaryGate(LogicGate):
    
    def __init__(self, n):
        LogicGate.__init__(self, n)
        
        self.pinA = None
        self.pinB = None
        
    def getPinA(self):
        if self.pinA == None:
            return input("Enter Pin A input for gate " + self.getName()+"-->")
        else:
            return self.pinA.getFrom().getOutput()
    
    def getPinB(self):
        if self.pinB == None:
            return input("Enter Pin B input for gate " + self.getName()+"-->")
        else:
            return self.pinB.getFrom().getOutput()
    
    def setNextPin(self,source):
        if self.pinA == None:
            self.pinA = source
        else:
            if self.pinB == None:
                self.pinB = source
            else:
                raise RuntimeError("Error: NO EMPTY PINS")

class UnaryGate(LogicGate):
    
    def __init__(self, n):
        LogicGate.__init__(self, n)
        
        self.pin = None
        
    def getPin(self):
        if self.pin == None:
            return int(input('Enter Pin input for gate ' + self.getName() + ' -->'))
        else:
            return self.pin.getFrom().getOutput()
    
    def setNextPin(self,source):
        if self.pin == None:
            self.pin = source
        else:
            raise RuntimeError("Error: NO EMPTY PINS")

## Individual Specific gates
These gates only reall need the performGateLogic() method to be specified. All other attributes/methods are recycled from the parents

In [3]:
class AndGate(BinaryGate):
    
    def __init__(self, n):
        BinaryGate.__init__(self, n)
        
    def performGateLogic(self):
        
        a = int(self.getPinA())
        b = int(self.getPinB())
        
        #print(a)
        #print(b)
        #print((a==1) and (b==1))
        return 1 if (a==1) and (b==1) else 0
    
class OrGate(BinaryGate):
    
    def __init__(self, n):
        BinaryGate.__init__(self, n)
        
    def performGateLogic(self):
        
        a = int(self.getPinA())
        b = int(self.getPinB())
        
        return 0 if (a==0) and (b==0) else 1
    
class NotGate(UnaryGate):
    
    def __init__(self, n):
        UnaryGate.__init__(self, n)
        
    def performGateLogic(self):
        
        a = int(self.getPin())
        
        return 0 if a==1 else 1
    
class NandGate(BinaryGate):
    
    def __init__(self, n):
        BinaryGate.__init__(self, n)
        
    def performGateLogic(self):
        
        a = int(self.getPinA())
        b = int(self.getPinB())
        
        return 0 if (a==1) and (b==1) else 1

class NorGate(BinaryGate):
    
    def __init__(self, n):
        BinaryGate.__init__(self, n)
        
    def performGateLogic(self):
        
        a = int(self.getPinA())
        b = int(self.getPinB())
        
        return 1 if (a==0) and (b==0) else 0

class XorGate(BinaryGate):
    
    def __init__(self, n):
        BinaryGate.__init__(self, n)
        
    def performGateLogic(self):
        
        a = int(self.getPinA())
        b = int(self.getPinB())
        
        return 0 if a==b else 1

class TermGate(UnaryGate):
    
    def __init__(self, n):
        UnaryGate.__init__(self, n)
        self.data = None
        
    def performGateLogic(self):
        if self.data == None:
            self.data = int(self.getPin())
        
        return self.data

## Connector, connects 2 gates

In [4]:
class Connector:

    def __init__(self, fgate, tgate):
        self.fromgate = fgate
        self.togate = tgate

        tgate.setNextPin(self)

    def getFrom(self):
        return self.fromgate

    def getTo(self):
        return self.togate

## Simple example use gates are used to add 2 one-bit numbers
The return is a list [S, C] where the answer of adding x + y is 2*C+S. Here C stands for carry

In [5]:
def half_adder():
    g1 = AndGate('And')
    g2 = XorGate('Xor')
    g3 = TermGate('Input 1')
    g4 = TermGate('Input 2')
    Connector(g3, g1)
    Connector(g3, g2)
    Connector(g4, g1)
    Connector(g4, g2)
    
    C = g1.getOutput()
    S = g2.getOutput()
    
    return [S, C]


In [76]:
half_adder()

Enter Pin input for gate Input 1 -->1
Enter Pin input for gate Input 2 -->1


[0, 1]

## Slightly more sophiscated 8-bit adder

In [14]:
def eight_bit_adder():
    l1gates = [[],[]]
    l2gates = []
    terms = [[], []]
    ans = []
    
    for i in range(8):
        terms[0].append(TermGate('{}-th bit of 1st number'.format(i)))
        terms[1].append(TermGate('{}-th bit of 2nd number'.format(i)))
        l1gates[0].append(XorGate('XorGate {}'.format(i)))
        l1gates[1].append(AndGate('AndGate {}'.format(i)))
        
        for a in range(2):
            for b in range(2):
                Connector(terms[a][i], l1gates[b][i])
    
    ans.append(l1gates[0][0].getOutput())
    
    for i in range(7):
        l2gates.append(XorGate('{}-th bit of outer later'.format(i)))
        Connector(l1gates[1][i], l2gates[i])
        Connector(l1gates[0][i+1], l2gates[i])
    
    for i in range(7):
        ans.append(l2gates[i].getOutput())
    
    ans.append(l1gates[1][7].getOutput())
    
    return ans


In [25]:
def lst_to_num(lst):
    ans = 0
    
    for i in range(len(lst)):
        ans += (2 ** i) * lst[i]
    
    return ans

print(lst_to_num(eight_bit_adder()))
print(lst_to_num(eight_bit_adder()))

Enter Pin input for gate 0-th bit of 1st number -->1
Enter Pin input for gate 0-th bit of 2nd number -->1
Enter Pin input for gate 1-th bit of 1st number -->1
Enter Pin input for gate 1-th bit of 2nd number -->1
Enter Pin input for gate 2-th bit of 1st number -->0
Enter Pin input for gate 2-th bit of 2nd number -->0
Enter Pin input for gate 3-th bit of 1st number -->0
Enter Pin input for gate 3-th bit of 2nd number -->0
Enter Pin input for gate 4-th bit of 1st number -->0
Enter Pin input for gate 4-th bit of 2nd number -->0
Enter Pin input for gate 5-th bit of 1st number -->0
Enter Pin input for gate 5-th bit of 2nd number -->0
Enter Pin input for gate 6-th bit of 1st number -->0
Enter Pin input for gate 6-th bit of 2nd number -->0
Enter Pin input for gate 7-th bit of 1st number -->0
Enter Pin input for gate 7-th bit of 2nd number -->0
6
Enter Pin input for gate 0-th bit of 1st number -->1
Enter Pin input for gate 0-th bit of 2nd number -->1
Enter Pin input for gate 1-th bit of 1st num